# 📘 Agentic Architectures 8: Episodic + Semantic Memory Stack

Welcome to the eighth notebook in our series. Today, we're tackling one of the most critical challenges in creating truly intelligent, long-term assistants: **persistent memory**. Standard chatbot memory is ephemeral, lasting only for a single session. To build a personalized agent that learns and grows with a user, we need a more robust solution.

We will implement a structured memory architecture that mirrors human cognition, combining two distinct types of memory:

1.  **Episodic Memory:** This is the memory of specific events or past interactions. It answers the question, "What happened?" (e.g., "Last week, the user asked me about NVIDIA's stock price."). We'll use a **vector database** for this to find past conversations relevant to the current topic.
2.  **Semantic Memory:** This is the memory of structured facts, concepts, and relationships extracted from those events. It answers the question, "What do I know?" (e.g., "User Alex is a conservative investor.", "Alex is interested in Tech Stocks."). We'll use a **graph database (Neo4j)** for this, as it excels at managing and querying complex relationships.

By combining these, our agent can not only recall past conversations but also build a rich, interconnected knowledge base about the user and the world, leading to deeply personalized and context-aware interactions.

### Definition
An **Episodic + Semantic Memory Stack** is an agent architecture that maintains two types of long-term memory. **Episodic memory** stores a chronological log of experiences (e.g., chat history summaries) and is typically searched based on semantic similarity. **Semantic memory** stores extracted, structured knowledge (facts, entities, relationships) in a knowledge base, often a graph.

### High-level Workflow

1.  **Interaction:** The agent has a conversation with the user.
2.  **Memory Retrieval (Recall):** For a new user query, the agent first queries both memory systems.
    *   It searches the **episodic** vector store for similar past conversations.
    *   It queries the **semantic** graph database for entities and facts related to the query.
3.  **Augmented Generation:** The retrieved memories are added to the prompt's context, allowing the LLM to generate a response that is aware of past interactions and learned facts.
4.  **Memory Creation (Encoding):** After the interaction is complete, a background process analyzes the conversation.
    *   It creates a concise summary of the turn (the new **episodic** memory).
    *   It extracts key entities and relationships (the new **semantic** memory).
5.  **Memory Storage:** The new episodic summary is embedded and saved to the vector store. The new semantic facts are written as nodes and edges in the graph database.

### When to Use / Applications
*   **Long-Term Personal Assistants:** An assistant that remembers your preferences, projects, and personal details over weeks or months.
*   **Personalized Systems:** E-commerce bots that remember your style, or educational tutors that remember your learning progress and weak spots.
*   **Complex Research Agents:** An agent that builds a knowledge graph of a topic as it explores documents, allowing it to answer complex, multi-hop questions.

### Strengths & Weaknesses
*   **Strengths:**
    *   **True Personalization:** Enables context and learning that persists indefinitely, far beyond a single session's context window.
    *   **Rich Understanding:** A graph database allows the agent to understand and reason about complex relationships between entities.
*   **Weaknesses:**
    *   **Complexity:** This is a significantly more complex architecture to build and maintain than a simple stateless agent.
    *   **Memory Bloat & Pruning:** Over time, the memory stores can become massive. Strategies for summarizing, consolidating, or pruning old/irrelevant memories are essential for long-term performance.

## Phase 0: Foundation & Setup

We'll install all necessary libraries, including drivers for our vector and graph databases, and configure our API keys.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community langchain-openai neo4j faiss-cpu tiktoken

In [2]:
import os
import uuid
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius, NebiusEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Memory Stack (Nebius)"

# Check for required environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Building the Memory Components

This is the core of our architecture. We'll define the structures for our memories and set up the connections to our databases. We'll also create the "Memory Maker" agent responsible for processing conversations and creating new memories.

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
embeddings = NebiusEmbeddings()

# --- 1. Vector Store for Episodic Memory ---
# In a real application, you'd persist this. For this example, it's in-memory.
try:
    episodic_vector_store = FAISS.from_texts(["Initial document to bootstrap the store"], embeddings)
except ImportError:
    console.print("[bold red]FAISS not installed. Please run `pip install faiss-cpu`.")[/bold red]
    episodic_vector_store = None

# --- 2. Graph DB for Semantic Memory ---
try:
    graph = Neo4jGraph(
        url=os.environ.get("NEO4J_URI"),
        username=os.environ.get("NEO4J_USERNAME"),
        password=os.environ.get("NEO4J_PASSWORD")
    )
    # Clear the graph for a clean run
    graph.query("MATCH (n) DETACH DELETE n")
except Exception as e:
    console.print(f"[bold red]Failed to connect to Neo4j: {e}. Please check your credentials and connection.[/bold red]")
    graph = None

# --- 3. Pydantic Models for the "Memory Maker" ---
# Define the structure of knowledge we want to extract.
class Node(BaseModel):
    id: str = Field(description="Unique identifier for the node, which can be a person's name, a company ticker, or a concept.")
    type: str = Field(description="The type of the node (e.g., 'User', 'Company', 'InvestmentPhilosophy').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the node.")

class Relationship(BaseModel):
    source: Node = Field(description="The source node of the relationship.")
    target: Node = Field(description="The target node of the relationship.")
    type: str = Field(description="The type of the relationship (e.g., 'IS_A', 'INTERESTED_IN').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the relationship.")

class KnowledgeGraph(BaseModel):
    """Represents the structured knowledge extracted from a conversation."""
    relationships: List[Relationship] = Field(description="A list of relationships to be added to the knowledge graph.")

# --- 4. The "Memory Maker" Agent ---
def create_memories(user_input: str, assistant_output: str):
    conversation = f"User: {user_input}\nAssistant: {assistant_output}"
    
    # 4a. Create Episodic Memory (Summarization)
    console.print("--- Creating Episodic Memory (Summary) ---")
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a summarization expert. Create a concise, one-sentence summary of the following user-assistant interaction. This summary will be used as a memory for future recall."),
        ("human", "Interaction:\n{interaction}")
    ])
    summarizer = summary_prompt | llm
    episodic_summary = summarizer.invoke({"interaction": conversation}).content
    
    new_doc = Document(page_content=episodic_summary, metadata={"created_at": uuid.uuid4().hex})
    episodic_vector_store.add_documents([new_doc])
    console.print(f"[green]Episodic memory created:[/green] '{episodic_summary}'")
    
    # 4b. Create Semantic Memory (Fact Extraction)
    console.print("--- Creating Semantic Memory (Graph) ---")
    extraction_llm = llm.with_structured_output(KnowledgeGraph)
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledge extraction expert. Your task is to identify key entities and their relationships from a conversation and model them as a graph. Focus on user preferences, goals, and stated facts."),
        ("human", "Extract all relationships from this interaction:\n{interaction}")
    ])
    extractor = extraction_prompt | extraction_llm
    try:
        kg_data = extractor.invoke({"interaction": conversation})
        if kg_data.relationships:
            for rel in kg_data.relationships:
                graph.add_graph_documents([rel], include_source=True)
            console.print(f"[green]Semantic memory created:[/green] Added {len(kg_data.relationships)} relationships to the graph.")
        else:
            console.print("[yellow]No new semantic memories identified in this interaction.[/yellow]")
    except Exception as e:
        console.print(f"[red]Could not extract or save semantic memory: {e}[/red]")

if episodic_vector_store and graph:
    print("Memory components initialized successfully.")

Memory components initialized successfully.


## Phase 2: The Memory-Augmented Agent

Now we'll build the agent that uses this memory system. We'll use LangGraph to define a clear, stateful workflow: retrieve memories, generate a response using those memories, and finally, update the memory with the latest interaction.

In [4]:
# Define the state for our LangGraph agent
class AgentState(TypedDict):
    user_input: str
    retrieved_memories: Optional[str]
    generation: str

# Define the nodes of the graph

def retrieve_memory(state: AgentState) -> Dict[str, Any]:
    """Node that retrieves memories from both episodic and semantic stores."""
    console.print("--- Retrieving Memories ---")
    user_input = state['user_input']
    
    # Retrieve from episodic memory
    retrieved_docs = episodic_vector_store.similarity_search(user_input, k=2)
    episodic_memories = "\n".join([doc.page_content for doc in retrieved_docs])
    
    # Retrieve from semantic memory
    # This is a simple retrieval; more advanced would involve entity extraction from the query
    try:
        graph_schema = graph.get_schema
        # Using a fulltext index for better retrieval. Neo4j automatically creates one on node properties.
        # A more robust solution might involve extracting entities from user_input first.
        semantic_memories = str(graph.query("""
            UNWIND $keywords AS keyword
            CALL db.index.fulltext.queryNodes("entity", keyword) YIELD node, score
            MATCH (node)-[r]-(related_node)
            RETURN node, r, related_node LIMIT 5
            """, {'keywords': user_input.split()}))
    except Exception as e:
        semantic_memories = f"Could not query graph: {e}"
        
    retrieved_content = f"Relevant Past Conversations (Episodic Memory):\n{episodic_memories}\n\nRelevant Facts (Semantic Memory):\n{semantic_memories}"
    console.print(f"[cyan]Retrieved Context:\n{retrieved_content}[/cyan]")
    
    return {"retrieved_memories": retrieved_content}

def generate_response(state: AgentState) -> Dict[str, Any]:
    """Node that generates a response using the retrieved memories."""
    console.print("--- Generating Response ---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful and personalized financial assistant. Use the retrieved memories to inform your response and tailor it to the user. If the memories indicate a user's preference (e.g., they are a conservative investor), you MUST respect it."),
        ("human", "My question is: {user_input}\n\nHere are some memories that might be relevant:\n{retrieved_memories}")
    ])
    generator = prompt | llm
    generation = generator.invoke(state).content
    console.print(f"[green]Generated Response:\n{generation}[/green]")
    return {"generation": generation}

def update_memory(state: AgentState) -> Dict[str, Any]:
    """Node that updates the memory with the latest interaction."""
    console.print("--- Updating Memory ---")
    create_memories(state['user_input'], state['generation'])
    return {}

# Build the graph
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve_memory)
workflow.add_node("generate", generate_response)
workflow.add_node("update", update_memory)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", "update")
workflow.add_edge("update", END)

memory_agent = workflow.compile()
print("Memory-augmented agent graph compiled successfully.")

Memory-augmented agent graph compiled successfully.


## Phase 3: Demonstration & Inspection

Let's see the agent in action. We'll simulate a multi-turn conversation. The first two turns will seed the memory. The third turn will test if the agent can use that memory for a personalized response. Finally, we'll directly inspect the databases to see the memories that were created.

In [5]:
def run_interaction(query: str):
    result = memory_agent.invoke({"user_input": query})
    return result['generation']

console.print("\n--- 💬 INTERACTION 1: Seeding Memory ---")
run_interaction("Hi, my name is Alex. I'm a conservative investor, and I'm mainly interested in established tech companies.")

console.print("\n--- 💬 INTERACTION 2: Asking a specific question ---")
run_interaction("What do you think about Apple (AAPL)?")

console.print("\n--- 🧠 INTERACTION 3: THE MEMORY TEST ---")
run_interaction("Based on my goals, what's a good alternative to that stock?")


--- 💬 INTERACTION 1: Seeding Memory ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Hello, Alex! It's great to meet you. As a conservative investor, focusing on established tech companies with strong fundamentals is a very sound strategy. I can certainly help you navigate that space. What's on your mind today?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user, Alex, introduced himself as a conservative investor interested in established tech companies.'
--- Creating Semantic Memory (Graph) ---
Semantic memory created: Added 2 relationships to the graph.



--- 💬 INTERACTION 2: Asking a specific question ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Apple (AAPL) is often considered a cornerstone for conservative tech portfolios. It has a massive market capitalization, a very strong brand, consistent profitability, and a history of returning value to shareholders through dividends and buybacks. Its ecosystem of products creates a loyal customer base, which provides a stable revenue stream. For a conservative investor, it generally aligns well with the goal of capital preservation while still offering growth potential. Would you like a deeper dive into its recent performance or financials?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user inquired about Apple (AAP


--- 🧠 INTERACTION 3: THE MEMORY TEST ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.

Relevant Facts (Semantic Memory):
[{'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'HAS_GOAL', 'properties': {}}, 'related_node': {'type': 'InvestmentPhilosophy', 'id': 'Conservative Investing', 'properties': {}}}, {'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'INTERESTED_IN', 'properties': {}}, 'related_node': {'type': 'Sector', 'id': 'Tech', 'properties': {}}}]
--- Generating Response ---
Generated Response:
Of course. Based on your stated goal of conservative investing in the tech sector, a great alternative to Apple (AAPL) would be Microsoft (MSFT).

Here's why it fits your profile

### Inspecting the Memory Stores

Let's look under the hood. We can query our databases directly to see the memories our agent created.

In [6]:
console.print("--- 🔍 Inspecting Episodic Memory (Vector Store) ---")
# We'll do a similarity search for a general concept to see what comes up
retrieved_docs = episodic_vector_store.similarity_search("User's investment strategy", k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content}")

console.print("\n--- 🕸️ Inspecting Semantic Memory (Graph Database) ---")
print(f"Graph Schema:\n{graph.get_schema}")

# Cypher query to see who is interested in what
query_result = graph.query("MATCH (n:User)-[r:INTERESTED_IN|HAS_GOAL]->(m) RETURN n, r, m")
print(f"Relationships in Graph:\n{query_result}")

--- 🔍 Inspecting Episodic Memory (Vector Store) ---


1. The user, Alex, introduced himself as a conservative investor interested in established tech companies.
2. Based on the user's conservative investment goals, the assistant suggested Microsoft (MSFT) as a good alternative to Apple (AAPL), highlighting its diversification and enterprise focus.
3. The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.



--- 🕸️ Inspecting Semantic Memory (Graph Database) ---


Graph Schema:
{'node_props': {'InvestmentPhilosophy': [{'property': 'id', 'type': 'STRING'}], 'Company': [{'property': 'id', 'type': 'STRING'}], 'Sector': [{'property': 'id', 'type': 'STRING'}], 'User': [{'property': 'id', 'type': 'STRING'}]}, 'rel_props': {}, 'relationships': [{'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Sector'}, {'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Company'}, {'start': 'User', 'type': 'HAS_GOAL', 'end': 'InvestmentPhilosophy'}]}

Relationships in Graph:
[{'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Conservative Investing', 'type': 'InvestmentPhilosophy'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Tech', 'type': 'Sector'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'AAPL', 'type': 'Company'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'MSFT', 'type': 'Company'}}]


## Conclusion

In this notebook, we successfully built an agent with a sophisticated, long-term memory system. The demonstration clearly shows the power of this architecture:

- **Stateless Failure:** A standard agent, when asked "Based on my goals, what's a good alternative?", would fail because it has no memory of the user's goals.
- **Memory-Augmented Success:** Our agent succeeded because it could:
    1.  **Recall Episodically:** It retrieved the summary of the first conversation: "The user, Alex, introduced himself as a conservative investor..."
    2.  **Recall Semantically:** It queried the graph and found the structured fact: `(User: Alex) -[HAS_GOAL]-> (InvestmentPhilosophy: Conservative)`.
    3.  **Synthesize:** It used this combined context to provide a highly relevant and personalized recommendation (Microsoft), explicitly referencing the user's conservative goals.

This combination of recalling *what happened* (episodic) and *what is known* (semantic) is a powerful paradigm for moving beyond simple, transactional agents to create true, learning companions. While managing this memory at scale presents challenges like pruning and consolidation, the foundational architecture we've built here is a significant step toward more intelligent and personalized AI systems.